# Prepare data

## Import libraries

In [1]:
import numpy as np
import pandas as pd

## Read datasets

In [2]:
awards_df = pd.read_csv('../data/awards.csv')
awards_df.head()

,year,winner,nominee,movie,wiki_url,award,category
0,1980,True,Robert De Niro,Raging Bull,/wiki/Raging_Bull,oscar,actor
1,1980,False,Robert Duvall,The Great Santini,/wiki/The_Great_Santini,oscar,actor
2,1980,False,John Hurt,The Elephant Man,/wiki/The_Elephant_Man_(film),oscar,actor
3,1980,False,Jack Lemmon,Tribute,/wiki/Tribute_(1980_film),oscar,actor
4,1980,False,Peter O'Toole,The Stunt Man,/wiki/The_Stunt_Man,oscar,actor


In [3]:
movies_df = pd.read_csv('../data/movies.csv')
movies_df.head()

,movie,year,success,imdb_id,rated,runtime,genres,director,writer,cast,language,country,imdb_score,imdb_votes,rt_score,metascore,box_office,other_win,other_nom
0,Raging Bull,1980,True,tt0081398,R,129 min,"Biography, Drama, Sport",Martin Scorsese,"Jake LaMotta (based on the book by), Joseph Ca...","Robert De Niro, Cathy Moriarty, Joe Pesci, Fra...",English,USA,8.2,"326,193",93,89.0,"$23,383,987",22.0,28.0
1,The Great Santini,1980,True,tt0079239,PG,115 min,Drama,Lewis John Carlino,"Pat Conroy (novel), Lewis John Carlino (writte...","Robert Duvall, Blythe Danner, Michael O'Keefe,...",English,USA,7.2,"5,756",95,64.0,"$4,702,575",3.0,4.0
2,The Elephant Man,1980,True,tt0080678,PG,124 min,"Biography, Drama",David Lynch,"Christopher De Vore (screenplay), Eric Bergren...","Anthony Hopkins, John Hurt, Anne Bancroft, Joh...","English, French","USA, UK",8.1,"224,116",92,78.0,"$26,010,864",10.0,14.0
3,Tribute,1980,True,tt0081656,PG,121 min,"Comedy, Drama",Bob Clark,"Bernard Slade (play), Bernard Slade (screenplay)","Jack Lemmon, Robby Benson, Lee Remick, Colleen...",English,Canada,6.2,642,55,NaN,"$9,000,000",2.0,13.0
4,The Stunt Man,1980,True,tt0081568,R,131 min,"Action, Comedy, Drama, Romance, Thriller",Richard Rush,"Lawrence B. Marcus (screenplay), Richard Rush ...","Peter O'Toole, Steve Railsback, Barbara Hershe...",English,USA,7.0,"8,848",90,77.0,"$7,063,886",4.0,8.0


## Prepare movies dataset

### Show general information

In [4]:
movies_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 499 entries, 0 to 498
Data columns (total 19 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   movie       499 non-null    object 
 1   year        499 non-null    int64  
 2   success     499 non-null    bool   
 3   imdb_id     492 non-null    object 
 4   rated       477 non-null    object 
 5   runtime     487 non-null    object 
 6   genres      492 non-null    object 
 7   director    486 non-null    object 
 8   writer      481 non-null    object 
 9   cast        492 non-null    object 
 10  language    491 non-null    object 
 11  country     492 non-null    object 
 12  imdb_score  485 non-null    float64
 13  imdb_votes  485 non-null    object 
 14  rt_score    479 non-null    object 
 15  metascore   451 non-null    float64
 16  box_office  465 non-null    object 
 17  other_win   471 non-null    float64
 18  other_nom   471 non-null    float64
dtypes: bool(1), float64(4), int64

### Delete insignificant columns

In [5]:
movies_df = movies_df.drop(columns=['cast', 'writer', 'country', 'success', 'imdb_id'], axis=1)

### Delete null values

In [6]:
movies_df.isnull().sum()

movie          0
year           0
rated         22
runtime       12
genres         7
director      13
language       8
imdb_score    14
imdb_votes    14
rt_score      20
metascore     48
box_office    34
other_win     28
other_nom     28
dtype: int64

In [7]:
movies_df = movies_df.fillna(value={'rated': 'Not Rated'})

In [8]:
movies_df.dropna(inplace=True)
print('Shape after cleaning:', movies_df.shape)

Shape after cleaning: (436, 14)


### Prettify and normalize numeric values (scores, votes)

#### Parse numbers

In [9]:
movies_df['rt_score'] = movies_df.rt_score.map(lambda x: x.split('/')[0])
movies_df['imdb_votes'] = movies_df.imdb_votes.map(lambda x: x.replace(',', ''))
movies_df['runtime'] = movies_df.runtime.map(lambda x: x.split()[0])
movies_df['box_office'] = movies_df.box_office.map(lambda x: x[1:].replace(',', ''))

#### Set dtypes

In [10]:
movies_df = movies_df.astype({
    'rt_score': float, 
    'metascore': float, 
    'imdb_score': float,
    'imdb_votes': int,
    'runtime': int,
    'box_office': int,
    'other_win': int,
    'other_nom': int,
})

#### Normalize scores

In [11]:
movies_df['rt_score'] /= 10
movies_df['metascore'] /= 10

#### Check result

In [12]:
movies_df.dtypes

movie          object
year            int64
rated          object
runtime         int64
genres         object
director       object
language       object
imdb_score    float64
imdb_votes      int64
rt_score      float64
metascore     float64
box_office      int64
other_win       int64
other_nom       int64
dtype: object

### Prettify categorical values

#### Remove unpleasant characters

In [13]:
movies_df['director'] = movies_df.director.map(lambda x: x.replace('(co-director)', '').strip())
movies_df['language'] = movies_df.language.map(lambda x: x.split(',')[0].strip())

#### Prepare genres

In [14]:
def prepare_genres(string: str) -> list[str]:
    genres = string.split(', ')
    return [*map(lambda x: x.lower().replace('-', ''), genres)]

movies_df['genres'] = movies_df.genres.map(prepare_genres)

In [15]:
genres = set()

for genres_row in movies_df['genres']:
    genres.update(genres_row)

In [16]:
print(f'There are {len(genres)} genres in total')
genres

There are 20 genres in total


{'action',
 'adventure',
 'animation',
 'biography',
 'comedy',
 'crime',
 'drama',
 'family',
 'fantasy',
 'history',
 'horror',
 'music',
 'musical',
 'mystery',
 'romance',
 'scifi',
 'sport',
 'thriller',
 'war',
 'western'}

In [17]:
genres_dicts = []

for idx, row in movies_df.iterrows():
    genre_row = {'movie': row.movie, 'year': row.year}    
    for genre in genres:
        genre_row[genre] = int(genre in row['genres'])
    genres_dicts.append(genre_row)

genres_df = pd.DataFrame.from_dict(genres_dicts)

In [18]:
genres_df.head()

,movie,year,history,crime,adventure,musical,action,war,western,scifi,...,music,fantasy,family,thriller,drama,biography,romance,animation,sport,horror
0,Raging Bull,1980,0,0,0,0,0,0,0,0,...,0,0,0,0,1,1,0,0,1,0
1,The Great Santini,1980,0,0,0,0,0,0,0,0,...,0,0,0,0,1,0,0,0,0,0
2,The Elephant Man,1980,0,0,0,0,0,0,0,0,...,0,0,0,0,1,1,0,0,0,0
3,The Stunt Man,1980,0,0,0,0,1,0,0,0,...,0,0,0,1,1,0,1,0,0,0
4,On Golden Pond,1981,0,0,0,0,0,0,0,0,...,0,0,0,0,1,0,0,0,0,0


#### Merge movies with genres dataframe

In [19]:
movies_df = movies_df.drop('genres', axis=1)
movies_df = pd.merge(movies_df, genres_df, on=['movie', 'year'])
movies_df.head()

,movie,year,rated,runtime,director,language,imdb_score,imdb_votes,rt_score,metascore,...,music,fantasy,family,thriller,drama,biography,romance,animation,sport,horror
0,Raging Bull,1980,R,129,Martin Scorsese,English,8.2,326193,9.3,8.9,...,0,0,0,0,1,1,0,0,1,0
1,The Great Santini,1980,PG,115,Lewis John Carlino,English,7.2,5756,9.5,6.4,...,0,0,0,0,1,0,0,0,0,0
2,The Elephant Man,1980,PG,124,David Lynch,English,8.1,224116,9.2,7.8,...,0,0,0,0,1,1,0,0,0,0
3,The Stunt Man,1980,R,131,Richard Rush,English,7.0,8848,9.0,7.7,...,0,0,0,1,1,0,1,0,0,0
4,On Golden Pond,1981,PG,109,Mark Rydell,English,7.6,28225,9.3,6.8,...,0,0,0,0,1,0,0,0,0,0


### Check resulting movies dataframe

In [20]:
movies_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 436 entries, 0 to 435
Data columns (total 33 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   movie       436 non-null    object 
 1   year        436 non-null    int64  
 2   rated       436 non-null    object 
 3   runtime     436 non-null    int64  
 4   director    436 non-null    object 
 5   language    436 non-null    object 
 6   imdb_score  436 non-null    float64
 7   imdb_votes  436 non-null    int64  
 8   rt_score    436 non-null    float64
 9   metascore   436 non-null    float64
 10  box_office  436 non-null    int64  
 11  other_win   436 non-null    int64  
 12  other_nom   436 non-null    int64  
 13  history     436 non-null    int64  
 14  crime       436 non-null    int64  
 15  adventure   436 non-null    int64  
 16  musical     436 non-null    int64  
 17  action      436 non-null    int64  
 18  war         436 non-null    int64  
 19  western     436 non-null    i

## Prepare awards dataset

### Show general information

In [21]:
awards_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3051 entries, 0 to 3050
Data columns (total 7 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   year      3051 non-null   int64 
 1   winner    3051 non-null   bool  
 2   nominee   3051 non-null   object
 3   movie     3051 non-null   object
 4   wiki_url  3044 non-null   object
 5   award     3051 non-null   object
 6   category  3051 non-null   object
dtypes: bool(1), int64(1), object(5)
memory usage: 146.1+ KB


### Delete insignificant columns

In [22]:
awards_df = awards_df.drop('wiki_url', axis=1)

### Clear null values

In [23]:
awards_df.isnull().sum() # it's already alright

year        0
winner      0
nominee     0
movie       0
award       0
category    0
dtype: int64

### Prettify string values and clear data

#### Clear nominees names

In [24]:
awards_df['nominee'] = awards_df.nominee.map(lambda x: x.replace('(posthumous)', '').strip())

#### Delete records for skipped awards (BAFTA director 1985)

In [25]:
awards_df = awards_df[~awards_df.movie.str.startswith('no record of nominations nor winner')]

### Combine all awards within row

In [26]:
index = ['movie', 'year', 'category', 'nominee']

winners_df = pd.pivot_table(
    data=awards_df,
    values='winner', columns=['award'], 
    index=index, aggfunc=np.sum
).fillna(0).reset_index()

winners_df.head()

award,movie,year,category,nominee,bafta,globe,oscar
0,"'night, Mother",1986,actress,Anne Bancroft,0.0,0.0,0.0
1,(500) Days of Summer,2009,actor,Joseph Gordon-Levitt,0.0,0.0,0.0
2,(500) Days of Summer,2009,picture,(500) Days of Summer,0.0,0.0,0.0
3,101 Dalmatians,1996,actress,Glenn Close,0.0,0.0,0.0
4,12 Years a Slave,2013,actor,Chiwetel Ejiofor,1.0,0.0,0.0


### Filter only Oscar nominees

In [27]:
columns = ['movie', 'year', 'category', 'nominee']
oscars_df = awards_df[awards_df['award'] == 'oscar'][columns]
awards_df = pd.merge(oscars_df, winners_df, on=columns)
awards_df.head()

,movie,year,category,nominee,bafta,globe,oscar
0,Raging Bull,1980,actor,Robert De Niro,0.0,1.0,1.0
1,The Great Santini,1980,actor,Robert Duvall,0.0,0.0,0.0
2,The Elephant Man,1980,actor,John Hurt,1.0,0.0,0.0
3,Tribute,1980,actor,Jack Lemmon,0.0,0.0,0.0
4,The Stunt Man,1980,actor,Peter O'Toole,0.0,0.0,0.0


### Set dtypes

In [28]:
awards_df = awards_df.astype({'bafta': int, 'globe': int, 'oscar': int})

### Check result

In [29]:
awards_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 856 entries, 0 to 855
Data columns (total 7 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   movie     856 non-null    object
 1   year      856 non-null    int64 
 2   category  856 non-null    object
 3   nominee   856 non-null    object
 4   bafta     856 non-null    int64 
 5   globe     856 non-null    int64 
 6   oscar     856 non-null    int64 
dtypes: int64(4), object(3)
memory usage: 53.5+ KB


## Combine awards and movies datasets

### Merge it

In [30]:
prepared_df = pd.merge(awards_df, movies_df, on=['movie', 'year'])
prepared_df.head()

,movie,year,category,nominee,bafta,globe,oscar,rated,runtime,director,...,music,fantasy,family,thriller,drama,biography,romance,animation,sport,horror
0,Raging Bull,1980,actor,Robert De Niro,0,1,1,R,129,Martin Scorsese,...,0,0,0,0,1,1,0,0,1,0
1,Raging Bull,1980,director,Martin Scorsese,0,0,0,R,129,Martin Scorsese,...,0,0,0,0,1,1,0,0,1,0
2,Raging Bull,1980,picture,Raging Bull,0,0,0,R,129,Martin Scorsese,...,0,0,0,0,1,1,0,0,1,0
3,The Great Santini,1980,actor,Robert Duvall,0,0,0,PG,115,Lewis John Carlino,...,0,0,0,0,1,0,0,0,0,0
4,The Elephant Man,1980,actor,John Hurt,1,0,0,PG,124,David Lynch,...,0,0,0,0,1,1,0,0,0,0


In [31]:
prepared_df.shape

(774, 38)

### Check for empty values

In [32]:
prepared_df.isnull().sum() # nice

movie         0
year          0
category      0
nominee       0
bafta         0
globe         0
oscar         0
rated         0
runtime       0
director      0
language      0
imdb_score    0
imdb_votes    0
rt_score      0
metascore     0
box_office    0
other_win     0
other_nom     0
history       0
crime         0
adventure     0
musical       0
action        0
war           0
western       0
scifi         0
comedy        0
mystery       0
music         0
fantasy       0
family        0
thriller      0
drama         0
biography     0
romance       0
animation     0
sport         0
horror        0
dtype: int64

### Save it

In [33]:
prepared_df.to_csv('../data/prepared.csv', index=False)